In [1]:
import os 
import time
import sys 
import json
from argparse import Namespace

from torch.utils.data import DataLoader

sys.path.append("./models/ExpansionNet_v2")
from projectCode.ModelLoader import loadModel
from projectCode.CocoDataset import CocoDataset
from projectCode.Perturbator import *
from projectCode.Evaluator import *

from projectCode.attack import Attacker

/home/captioning/miniconda3/envs/captioning/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/captioning/miniconda3/envs/captioning/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/captioning/miniconda3/envs/captioning/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out ht

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(device)

cuda


In [3]:
ann_file = "./annotations/annotations/captions_val2017.json"
num_examples = 256

seed = 42

ds = CocoDataset(ann_file, num_examples, img_dir = "imgs",seed = seed)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Selected 256 images.


In [4]:
model = loadModel("BLIP",model_path = None, device=device)

In [ ]:
corruption_types = [Perturbation(), 
                    Blur(), 
                    Noise(noise_level = 0.12),
                    Noise(noise_type = "Uniform",noise_level = 0.33),
                    Noise(noise_type ="Laplace",noise_level = 0.1), 
                    Patch(patch_size = (50,50))]
evaluator = Evaluator(model,ds,corruption_types,"BLEU",batch_size=64, seed=seed)

In [ ]:
filtered_img = []
filtered_cap = []
for img, cap in ds:
    p = model(img.unsqueeze(0))
    score = METEOR_eval([cap], p)
    if score[0] >= 0.45:
        filtered_img.append(img)
        filtered_cap.append(cap)

In [ ]:
filtered_img_tensor = torch.stack(filtered_img)

captions = {
    "captions": filtered_cap
}

file_name = "captions_filtered.json"
with open(file_name, 'w') as json_file:
    json.dump(captions, json_file)

file_name = "img_filtered"
torch.save(filtered_img_tensor, file_name)

In [ ]:
scores = evaluator.evaluate_dataset()

In [ ]:
scores_tensor = torch.tensor(scores)

result = {
    "seed": seed,
    "corruption_types": ['id','blur','normal noise','uniform noise','laplace noise','patch'],
    "scores": {
        "id": scores_tensor[0].mean().item(),
        "blur": scores_tensor[1].mean().item(),
        "normal noise": scores_tensor[2].mean().item(),
        "uniform noise": scores_tensor[3].mean().item(),
        "laplace noise": scores_tensor[4].mean().item(),
        "patch": scores_tensor[5].mean().item(),
        
    }
}

print(result)

In [ ]:
file_name = "BLIP_BLUP_256_BLEU_Blur_Noise_Patch.json"

with open(file_name, 'w') as json_file:
    json.dump(result, json_file)

print(f"Results have been saved to {file_name}")